In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.read_parquet('/kaggle/input/visual-taxonomy/category_attributes.parquet')
print(df.head())

In [ ]:
train_path = "/kaggle/input/visual-taxonomy/train.csv"
train_df = pd.read_csv(train_path)

df_men = train_df.loc[train_df['Category'] == 'Men Tshirts']
df_men.reset_index(drop=True, inplace=True)
df_men

In [ ]:
temp = train_df.groupby('Category').apply(len)

In [ ]:
print(temp)

In [ ]:
l = df['Attribute_list'][0]
l

In [ ]:
for item in l :
    print(item)

In [ ]:
from keras.utils import normalize, to_categorical
pd.options.mode.chained_assignment = None

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from tqdm import tqdm
from keras.preprocessing import image

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import json

# Initialize a dictionary to store mappings for each attribute
mappings = {}

# Iterate through each attribute column to create and save mappings
for attr in df_men.columns[3:8]:  # Adjust column range if necessary
    # Convert column to categorical and get the categories
    c = df_men[attr].astype('category')
    df_men[attr] = c.cat.codes  # Apply cat.codes to the DataFrame
    
    # Store the mapping of integer codes to original categories
    mappings[attr] = dict(enumerate(c.cat.categories))

# Display the mappings for verification
print("Mappings:")
for key, value in mappings.items():
    print(f"{key}: {value}")

# Save mappings to /kaggle/working/ for download
with open("/kaggle/working/attribute_mappings.json", "w") as f:
    json.dump(mappings, f, indent=4)

print("\nMappings saved to /kaggle/working/attribute_mappings.json")


In [ ]:

# df_men["attr_1"] = df_men["attr_1"].astype('category').cat.codes
# df_men["attr_2"] = df_men["attr_2"].astype('category').cat.codes
# df_men["attr_3"] = df_men["attr_3"].astype('category').cat.codes
# df_men["attr_4"] = df_men["attr_4"].astype('category').cat.codes
# df_men["attr_5"] = df_men["attr_5"].astype('category').cat.codes
y_color = np.array(df_men["attr_1"])
y_neck = np.array(df_men["attr_2"])
y_pattern = np.array(df_men["attr_3"])
y_print = np.array(df_men["attr_4"])
y_sleeve = np.array(df_men["attr_5"])
y_color = to_categorical(y_color)
y_neck = to_categorical(y_neck)
y_pattern = to_categorical(y_pattern)
y_print = to_categorical(y_print)
y_sleeve = to_categorical(y_sleeve)


In [ ]:
image_directory = "/kaggle/input/visual-taxonomy/train_images/"

In [ ]:
print(len(df))
print(len(y_color))

In [ ]:
SIZE=200
X_dataset = []  
for i in tqdm(range(df_men.shape[0])):
    prep = str(df_men['id'][i])
    prep = '0'*(6-len(prep)) + prep
    img = image.load_img(image_directory +prep+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    # img = img/255.
    img = preprocess_input(img)
    X_dataset.append(img)
X = np.array(X_dataset)

In [ ]:
IMAGE_SIZE = (200, 200, 3)


# Load the base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE)

# Freeze the base model: False
base_model.trainable = True

In [ ]:
base_model.summary()

In [ ]:
print(len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 150

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
import keras

In [ ]:

y_combined = np.hstack((y_color, y_neck, y_pattern, y_print, y_sleeve))

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_combined,
    test_size=0.2,
    random_state=42
)

In [ ]:
import gc

In [ ]:
del X
del train_df 
gc.collect()

In [ ]:
# color_classes = df_men['attr_1'].unique()
# neck_classes = df_men['attr_2'].unique()
# pattern_classes = df_men['attr_3'].unique()
# print_classes = df_men['attr_4'].unique()
# sleeve_classes = df_men['attr_5'].unique()

NUM_CLASSES_COLOR = len(df_men['attr_1'].unique())-1
NUM_CLASSES_NECK = len(df_men['attr_2'].unique())-1
NUM_CLASSES_PATTERN = len(df_men['attr_3'].unique())-1
NUM_CLASSES_PRINT = len(df_men['attr_4'].unique())-1
NUM_CLASSES_SLEEVE = len(df_men['attr_5'].unique())-1

In [ ]:
# Split the combined labels back into their respective arrays
train_color_labels = y_train[:, :NUM_CLASSES_COLOR]
train_neck_labels = y_train[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_NECK]
train_pattern_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_NECK:NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN]
train_print_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN:NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT]
train_sleeve_labels = y_train[:, -NUM_CLASSES_SLEEVE:]

val_color_labels = y_val[:, :NUM_CLASSES_COLOR]
val_neck_labels = y_val[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_NECK]
val_pattern_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_NECK:NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN]
val_print_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN:NUM_CLASSES_COLOR + NUM_CLASSES_NECK + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT]
val_sleeve_labels = y_val[:, -NUM_CLASSES_SLEEVE:]

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    horizontal_flip=True
)

In [ ]:
class MultiLabelDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size, datagen):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.datagen = datagen
        self.indices = np.arange(len(images))

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = self.images[batch_indices]
        batch_labels = {key: label[batch_indices] for key, label in self.labels.items()}
        
        # Apply data augmentation
        batch_images = np.array([self.datagen.random_transform(img) for img in batch_images])

        return batch_images, batch_labels

In [ ]:
train_labels = {
    'color': train_color_labels,
    'neck': train_neck_labels,
    'pattern': train_pattern_labels,
    'print': train_print_labels,
    'sleeve': train_sleeve_labels
}

batch_size = 32
train_generator = MultiLabelDataGenerator(X_train, train_labels, batch_size, train_datagen)

In [ ]:
# checkpoint_filepath = '/kaggle/working/checkpoint.model.keras'
# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor='val_color_accuracy',
#     mode='max',
#     save_best_only=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, name="macro_f1_score", **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Ensure consistency in data types
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(K.round(y_pred), tf.float32)

        # Calculate true positives, false positives, and false negatives
        self.tp.assign_add(K.sum(y_true * y_pred))
        self.fp.assign_add(K.sum((1 - y_true) * y_pred))
        self.fn.assign_add(K.sum(y_true * (1 - y_pred)))

    def result(self):
        precision = self.tp / (self.tp + self.fp + K.epsilon())
        recall = self.tp / (self.tp + self.fn + K.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1

    def reset_states(self):
        self.tp.assign(0)
        self.fp.assign(0)
        self.fn.assign(0)


In [ ]:
class AverageF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, patience=10, filepath='checkpoint.model.keras'):
        super(AverageF1Callback, self).__init__()
        self.patience = patience
        self.best_avg_f1 = -1
        self.wait = 0
        self.filepath = filepath

    def on_epoch_end(self, epoch, logs=None):
        # Compute the average macro F1 score across all outputs
        avg_f1 = np.mean([
            logs.get("val_color_macro_f1_score", 0),
            logs.get("val_neck_macro_f1_score", 0),
            logs.get("val_pattern_macro_f1_score", 0),
            logs.get("val_print_macro_f1_score", 0),
            logs.get("val_sleeve_macro_f1_score", 0)
        ])

        if avg_f1 > self.best_avg_f1:
            self.best_avg_f1 = avg_f1
            self.wait = 0
            # Save best model
            self.model.save(self.filepath)
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nEpoch {epoch}: early stopping")
                self.model.stop_training = True


In [ ]:
# from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

# def macro_f1_score(y_true, y_pred):
#     y_pred = K.round(y_pred)  # Convert probabilities to binary labels (0 or 1)
#     tp = K.sum(K.round(y_true * y_pred))  # True positives
#     fp = K.sum(K.round((1 - y_true) * y_pred))  # False positives
#     fn = K.sum(K.round(y_true * (1 - y_pred)))  # False negatives

#     precision = tp / (tp + fp + K.epsilon())
#     recall = tp / (tp + fn + K.epsilon())
#     f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
#     return f1

# Adjust the ModelCheckpoint to monitor the macro_f1_score across validation data
checkpoint_filepath = '/kaggle/working/checkpoint.model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_macro_f1_score',  # Use the custom F1 metric
    mode='max',
    save_best_only=True
)

In [ ]:
# Create a model with five output heads for each attribute
inputs = layers.Input(shape=IMAGE_SIZE)
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)

# Define output layers for each attribute
output_color = layers.Dense(NUM_CLASSES_COLOR, activation='softmax', name='color')(x)
output_neck = layers.Dense(NUM_CLASSES_NECK, activation='softmax', name='neck')(x)
output_pattern = layers.Dense(NUM_CLASSES_PATTERN, activation='softmax', name='pattern')(x)
output_print = layers.Dense(NUM_CLASSES_PRINT, activation='softmax', name='print')(x)
output_sleeve = layers.Dense(NUM_CLASSES_SLEEVE, activation='softmax', name='sleeve')(x)

# Define the model
model = models.Model(inputs=inputs, outputs=[output_color, output_neck, output_pattern, output_print, output_sleeve])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
    loss={
        'color': 'categorical_crossentropy',
        'neck': 'categorical_crossentropy',
        'pattern': 'categorical_crossentropy',
        'print': 'categorical_crossentropy',
        'sleeve': 'categorical_crossentropy'
    },
    metrics={
        'color': ['accuracy', MacroF1Score()],
        'neck': ['accuracy', MacroF1Score()],
        'pattern': ['accuracy', MacroF1Score()],
        'print': ['accuracy', MacroF1Score()],
        'sleeve': ['accuracy', MacroF1Score()]
    }
)
# Summary of the model
model.summary()

In [ ]:
# Instantiate the custom callback
average_f1_callback = AverageF1ScoreCallback(patience=10, filepath=checkpoint_filepath)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_avg_macro_f1_score',  # Using the custom F1 metric
    patience=10,
    mode='max',
    restore_best_weights=True
)


In [ ]:
# from keras.callbacks import ReduceLROnPlateau

# # Add this callback to your training
# reduce_lr = ReduceLROnPlateau(monitor='val_color_accuracy', factor=0.5, patience=3, min_lr=1e-7)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_avg_macro_f1_score',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    mode='max'
)

In [ ]:


history = model.fit(
    train_generator,  # Image data
    validation_data=(X_val, {
        'color': val_color_labels,
        'neck': val_neck_labels,
        'pattern': val_pattern_labels,
        'print': val_print_labels,
        'sleeve': val_sleeve_labels
    }),
    epochs=100,
    callbacks=[model_checkpoint_callback, early_stopping, reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output from model.fit()
# Example: history = model.fit(...)

# Plotting the accuracy
plt.figure(figsize=(12, 6))

# Plot training accuracy
plt.plot(history.history['color_accuracy'], label='Color Train Accuracy', color='blue')
plt.plot(history.history['neck_accuracy'], label='Neck Train Accuracy', color='orange')
plt.plot(history.history['pattern_accuracy'], label='Pattern Train Accuracy', color='green')
plt.plot(history.history['print_accuracy'], label='Print Train Accuracy', color='red')
plt.plot(history.history['sleeve_accuracy'], label='Sleeve Train Accuracy', color='purple')

# Plot validation accuracy
plt.plot(history.history['val_color_accuracy'], label='Color Validation Accuracy', linestyle='dashed', color='blue')
plt.plot(history.history['val_neck_accuracy'], label='Neck Validation Accuracy', linestyle='dashed', color='orange')
plt.plot(history.history['val_pattern_accuracy'], label='Pattern Validation Accuracy', linestyle='dashed', color='green')
plt.plot(history.history['val_print_accuracy'], label='Print Validation Accuracy', linestyle='dashed', color='red')
plt.plot(history.history['val_sleeve_accuracy'], label='Sleeve Validation Accuracy', linestyle='dashed', color='purple')

# Adding labels and title
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import os 

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'/kaggle/working/checkpoint.model.keras')